In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define your data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Rest of your code...

# Define your data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Load your training dataset
train_generator = train_datagen.flow_from_directory(
    'C:/Users/Legion/Desktop/AI/dataset/training_set',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

# Build your model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile your model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train your model
history = model.fit(train_generator, epochs=20)

# Save the trained model
model.save('Catanddog.h5')


In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the saved model
model = load_model('Catanddog.h5')  # Load your saved model

# Define the path to your test dataset
test_data_dir = 'C:/Users/Legion/Desktop/Project/AI-NeuralNetwork ClassficationD&C/dataset/test_set'

# Data augmentation and preprocessing for the test dataset
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess the test dataset
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150, 150),  # Ensure it matches the model's input size
    batch_size=32,
    class_mode='binary',  # Use 'binary' if it's a binary classification
    shuffle=False
)

# Evaluate the model on the test dataset
evaluation = model.evaluate(test_generator)

# Print the test accuracy
print(f'Test Accuracy: {evaluation[1] * 100:.2f}%')


Found 2000 images belonging to 2 classes.
63/63 [==============================] - 7s 102ms/step - loss: 0.3624 - accuracy: 0.8345
Test Accuracy: 83.45%


In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model('catanddog.h5')

# Create the main application window
app = tk.Tk()
app.title("My App")

# Create a frame to hold the image rows
image_frame = tk.Frame(app)
image_frame.pack(side="top")

# Create a list to store image paths
image_paths = []
image_labels = []

# Create a list to store ImageTk.PhotoImage objects for displaying images
image_thumbnails = []

# Create a variable to keep track of the current row
current_row = None

# Function to browse and select image files
def browse_images():
    global current_row  # Use the global current_row variable
    paths = filedialog.askopenfilenames(
        title="Select Image Files", filetypes=(("Image files", "*.jpg *.png"), ("all files", "*.*"))
    )

    if len(image_paths) + len(paths) > 30:
        show_full_images()
        return

    for path in paths:
        image_paths.append(path)
        image_labels.append("Not Classified")

    update_image_display()

# Function to classify images and update the result label
def classify_images():
    if not image_paths:
        result_label.config(text="Please select images first")
        return

    images = [np.array(Image.open(path).resize((150, 150))) for path in image_paths]
    normalized_images = np.array(images) / 255.0
    predictions = model.predict(normalized_images)

    num_cats = sum(predictions < 0.5)
    num_dogs = sum(predictions >= 0.5)

    result_label.config(text=f"Cat count: {num_cats}, Dog count: {num_dogs}")

    for i, prediction in enumerate(predictions):
        if prediction < 0.5:
            image_labels[i] = "Cat"
        else:
            image_labels[i] = "Dog"

    update_image_display()


# Function to update the image display
def update_image_display():
    global current_row  # Use the global current_row variable

    # Clear the current row frame
    if current_row:
        current_row.pack_forget()
        current_row.destroy()

    num_images_per_row = 10
    current_row = tk.Frame(image_frame)
    current_row.pack()

    for i in range(len(image_paths)):
        if i % num_images_per_row == 0:
            current_row = tk.Frame(image_frame)
            current_row.pack()

        path = image_paths[i]
        label = image_labels[i]

        img = Image.open(path)
        img.thumbnail((150, 150))
        photo = ImageTk.PhotoImage(img)
        label = tk.Label(current_row, image=photo, text=label, compound=tk.TOP)
        label.image = photo
        label.pack(side="left")
        image_thumbnails.append(label)

# Function to show all images at full size
def show_full_images():
    global current_row  # Use the global current_row variable
    result_label.config(text="Max limit of 30 images reached.")

    # Clear the current row frame
    if current_row:
        current_row.pack_forget()
        current_row.destroy()

    num_images_per_row = 10
    current_row = tk.Frame(image_frame)
    current_row.pack()

    for i in range(len(image_paths)):
        if i % num_images_per_row == 0:
            current_row = tk.Frame(image_frame)
            current_row.pack()

        path = image_paths[i]
        label = image_labels[i]

        img = Image.open(path)
        photo = ImageTk.PhotoImage(img)
        label = tk.Label(current_row, image=photo, text=label, compound=tk.TOP)
        label.image = photo
        label.pack(side="left")
        image_thumbnails.append(label)

# Function to reset all images
def reset_images():
    global image_paths, image_labels
    image_paths = []
    image_labels = []
    result_label.config(text="")
    update_image_display()


# Create a button to reset images
reset_button = tk.Button(app, text="Reset Images", command=reset_images)
reset_button.pack(side="bottom")


# Create a button to select images
select_button = tk.Button(app, text="Select Images", command=browse_images)
select_button.pack(side="bottom")

# Create a button to classify images
classify_button = tk.Button(app, text="Classify Images", command=classify_images)
classify_button.pack(side="bottom")

# Create a label to display the result
result_label = tk.Label(app, text="", font=("Helvetica", 16))
result_label.pack(side="bottom")

# Run the main application loop
app.mainloop()


1/1 [==============================] - 0s 187ms/step
